# FSK Synced

In [ ]:
from coders import BitStream, WavStream, rint, sync_padding, min_error
from coders import SimpleFSK as Coder
from pipeline.util import ffmpeg
import scipy.io.wavfile

coder = Coder()
ffmpeg = ffmpeg('ffmpeg')
noise_in = 0.15
noise_out = 0.15
sync_duration = 0.4

In [ ]:
input_source = BitStream(np.random.randint(2, size=256))
verification = input_source.assymbolwidth(coder.symbol_width)
print('Input:', input_source)
print('Verification:', verification)

In [ ]:
encoded = coder.encode(input_source)
sync = sync_padding(coder, sync_duration)
padded = np.concatenate((sync, encoded))
padded_noisy = np.clip(padded + np.random.normal(0, noise_in, len(padded)), -1, 1)
scipy.io.wavfile.write('test_in.wav', coder.r, padded_noisy)
ffmpeg('test_in.wav', 'test_temp.amr', 8000)
ffmpeg('test_temp.amr', 'test_out.wav', coder.r)
converted_rate, converted_raw = scipy.io.wavfile.read('test_out.wav')
converted_symbol_len = rint(converted_rate * coder.symbol_duration)
converted = converted_raw / 32768
converted_noisy = np.clip(converted + np.random.normal(0, noise_out, len(converted)), -1, 1)

In [ ]:
shift = (len(converted) - len(padded)) // 2
shifted = WavStream(converted_noisy[shift:len(padded)+shift], converted_rate, converted_symbol_len)
print('Shifted:', shift)

In [ ]:
sync_padding = rint(sync_duration * converted_rate)
sync_clip = sync_padding // 10
sync_shift = shift + min_error(padded, coder.filter(shifted), shift, sync_padding, sync_clip)
print('Sync shift:', sync_shift)

synced = WavStream(converted_noisy[sync_shift+sync_padding:len(encoded)+sync_shift+sync_padding],
                    converted_rate, converted_symbol_len)

In [ ]:
%time output = coder.decode(synced)

In [ ]:
check = verification == output
quality = (check.sum()/len(check)).item()
rate = len(synced)/(len(input_source.assymbolwidth(1)) * converted_rate)

start, end = 0, 2000
sns.tsplot(encoded[start:end])
sns.tsplot(synced[start:end], color='orange', alpha=.2)
sns.tsplot(coder.filter(synced)[start:end], color='r', alpha=.5)
print('Rate:', round(rate, 2))
print('Quality:', round(quality, 2))